### Install and Import Dependencies

In [ ]:
# !pip install mediapipe opencv-python 

In [1]:
from mediapipe import solutions as mp
import cv2
from google.protobuf.json_format import MessageToDict
import time
import os

### Load Fingers Images 

- These finger images will be shown on the webcam according to the prediction made. 

In [2]:
# Load finger images 
folder_path = "finger_images"
overlay_list = []

for img_path in os.listdir(folder_path):
    image = cv2.imread(f"{folder_path}/{img_path}")
    print(f"{folder_path}/{img_path}")
    overlay_list.append(image)
print(len(overlay_list))

finger_images/finger_0.png
finger_images/finger_1.png
finger_images/finger_2.png
finger_images/finger_3.png
finger_images/finger_4.png
finger_images/finger_5.png
6


### Finger Count Detector

To connect your phone camera:
- Install `DroidCam` App on your phone and `DroidCam Client` on your laptop. 
- Put the IP address in the `cv2.VideoCapture()` below.
- Or use other phone camera IP service provider.

In [4]:
# Using phone camera ip with DroidCam
# Webcam is 0
cap = cv2.VideoCapture("http://192.168.100.6:4747/mjpegfeed?640x480")
tip_id = [4, 8, 12, 16, 20]
pTime = 0

with mp.hands.Hands() as hands:

    while True:
        success, frame = cap.read()
        lm_list = []

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(image)

        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:
                for idx, lm in enumerate(hand_landmark.landmark):
                    h, w, c = image.shape
                    cx, cy = int(lm.x*w), int(lm.y*h)
                    lm_list.append([idx, cx, cy])
                        
                mp.drawing_utils.draw_landmarks(
                    image, hand_landmark, mp.hands.HAND_CONNECTIONS)
        
        # Detect handedness
        if results.multi_handedness:
            for idx, hand_handedness in enumerate(results.multi_handedness):
                handedness_dict = MessageToDict(hand_handedness)
                print(handedness_dict)        
        
        # Detect finger count
        threshold = 0.3
        if len(lm_list) != 0:
            fingers = []
            for id in tip_id:
                if id == tip_id[0]:
                    # When using front camera, the hand is mirrored
                    if results.multi_handedness == "Right":
                        print(results.multi_handedness)
                        if lm_list[id][1] > lm_list[id-2][1] + threshold*100:
                            fingers.append(1)
                            continue
                        else:
                            fingers.append(0)
                            continue
                    else:
                        if lm_list[id][1] < lm_list[id-2][1] - threshold*100:
                            fingers.append(1)
                            continue
                        else:
                            fingers.append(0)
                            continue
                    
                if lm_list[id][2] < lm_list[id-2][2] - threshold*100:
                    fingers.append(1)
                else:
                    fingers.append(0)
            print(fingers, end=" ")
            
            # work in progress# work in progress# work in progress# work in progress# work in progress
                
        # Display finger_image that is detected        
        h, w, c = overlay_list[0].shape
        image[0:h, 0:w] = overlay_list[0]
        # work in progress

        # Display framerate
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
        cv2.putText(image, f"FPS: {int(fps)}", (500, 50), cv2.FONT_HERSHEY_PLAIN,
                    2, (255, 0, 0), 2)
    
        # Recolor for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        cv2.imshow("Hand detection", image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

{'classification': [{'index': 0, 'score': 0.99994296, 'label': 'Left'}]}
[1, 0, 0, 0, 0] {'classification': [{'index': 0, 'score': 0.9089309, 'label': 'Left'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[1, 1, 0, 0, 0] {'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[0, 0, 0, 0, 0] {'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[1, 1, 0, 0, 0] {'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[1, 1, 0, 0, 0] {'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[1, 1, 0, 0, 0] {'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[1, 1, 0, 0, 0] {'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
{'classification'

[0, 0, 0, 0, 0] {'classification': [{'index': 1, 'score': 0.90966606, 'label': 'Right'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[0, 0, 0, 0, 0] {'classification': [{'index': 1, 'score': 0.99940777, 'label': 'Right'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[0, 0, 0, 0, 0] {'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[1, 1, 1, 0, 0] {'classification': [{'index': 0, 'score': 0.8405482, 'label': 'Left'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[0, 0, 0, 0, 0] {'classification': [{'index': 0, 'score': 0.94153005, 'label': 'Left'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[0, 0, 0, 0, 0] {'classification': [{'index': 0, 'score': 0.9618596, 'label': 'Left'}]}
{'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[0, 0, 0, 0, 0] {'classification': [{'index': 0, 'score': 1.0, 'label': 'Left'}]}
[1, 1, 1, 0, 0] {'classification': [{'index': 0, 'score': 0.9

<!-- ![image](https://user-images.githubusercontent.com/60457367/123659592-715aba80-d865-11eb-9acd-a15b719c2fe9.png) -->